# iris_data를 xgboost로 적용을 해보자.

In [1]:
#import packages
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris

#Loading iris dataset from sklearn
iris = load_iris()

#independent feautres
X = iris.data

# target features
y = iris.target

In [2]:
#import XGboost
from xgboost import XGBClassifier

#Defining XGB Classification model
clf = XGBClassifier()

- xgboost의 단점은 복잡한 하이퍼 파라미터가 있다.

# 1.Grid SearchCV
- 사용자가 하이퍼 파라미터마다 몇가지 값을 가진 리스트를 입력하면, 가능한 하이퍼 파라미터의 경우의 수마다 예측 성능을 측정하여 사용자가 일일이 하이퍼 파라미터를 설정하고, 예측 성능을 비교하여 최적의 파라미터를 찾는 수고를 줄이고 이 과정을 한꺼번에 진행한다.

In [3]:
#Importing packages from sklearn

from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics

#defining a set of values as a dictionary for hyperparameters

param_grid = {
    "n_estimators":[100,200,300,400],
    "max_depth":[1,3,5,7],
    "reg_lambda":[.01,.1,.5]    
}

#declaring GridSearchCV model

model = model_selection.GridSearchCV(
    estimator = clf,
    param_grid = param_grid,
    scoring = 'accuracy',
    verbose = 10,
    n_jobs = 1,
    cv = 5    
)

#fitting values to the gridsearchcv model

model.fit(X,y)
#printing the best possible values to enhance accuracy
print(model.best_params_)
print(model.best_estimator_)
#printing the best score
print(model.best_score_)
#import joblib
#joblib.dump(model, 'model_xgboost.pkl') q

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5; 1/48] START max_depth=1, n_estimators=100, reg_lambda=0.01.............
[CV 1/5; 1/48] END max_depth=1, n_estimators=100, reg_lambda=0.01;, score=0.967 total time=   1.3s
[CV 2/5; 1/48] START max_depth=1, n_estimators=100, reg_lambda=0.01.............
[CV 2/5; 1/48] END max_depth=1, n_estimators=100, reg_lambda=0.01;, score=0.967 total time=   0.0s
[CV 3/5; 1/48] START max_depth=1, n_estimators=100, reg_lambda=0.01.............
[CV 3/5; 1/48] END max_depth=1, n_estimators=100, reg_lambda=0.01;, score=0.900 total time=   0.0s
[CV 4/5; 1/48] START max_depth=1, n_estimators=100, reg_lambda=0.01.............
[CV 4/5; 1/48] END max_depth=1, n_estimators=100, reg_lambda=0.01;, score=0.933 total time=   0.0s
[CV 5/5; 1/48] START max_depth=1, n_estimators=100, reg_lambda=0.01.............
[CV 5/5; 1/48] END max_depth=1, n_estimators=100, reg_lambda=0.01;, score=0.967 total time=   0.0s
[CV 1/5; 2/48] START max_depth=1, n_es

# 2. RandomizedSearchCV

- 그리드 서치에서는 grid_param과 같이 매개변수마다 특정 값을 지정해주었습니다. 만약에 변수 범위가 너무 다양하다면 하나하나 작성해주는게 너무 힘들다.

- 하이퍼 파라미터 검색 반영이 너무 클때 사용하는 방식이 Randomized Search입니다.


In [4]:
#defining a set of values as a dictionary for hyperparameters

param_grid = {
    "n_estimators":[100,200,300,400],
    "max_depth":[1,3,5,7],
    "reg_lambda":[.01,.1,.5]    
}

#declaring RandomizedSearchCV model

model = model_selection.RandomizedSearchCV(
    estimator = clf,
    param_distributions = param_grid,
    scoring = 'accuracy',
    verbose = 10,
    n_jobs = 1,
    cv = 5,
    n_iter=10
)

#fitting values to the RandomizedSearchCV model

model.fit(X,y)

#printing the best possible values to enhance accuracy

print(model.best_params_)
print(model.best_estimator_)
#printing the best score
print(model.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START max_depth=1, n_estimators=300, reg_lambda=0.5..............
[CV 1/5; 1/10] END max_depth=1, n_estimators=300, reg_lambda=0.5;, score=0.967 total time=   0.3s
[CV 2/5; 1/10] START max_depth=1, n_estimators=300, reg_lambda=0.5..............
[CV 2/5; 1/10] END max_depth=1, n_estimators=300, reg_lambda=0.5;, score=0.967 total time=   0.5s
[CV 3/5; 1/10] START max_depth=1, n_estimators=300, reg_lambda=0.5..............
[CV 3/5; 1/10] END max_depth=1, n_estimators=300, reg_lambda=0.5;, score=0.900 total time=   0.7s
[CV 4/5; 1/10] START max_depth=1, n_estimators=300, reg_lambda=0.5..............
[CV 4/5; 1/10] END max_depth=1, n_estimators=300, reg_lambda=0.5;, score=0.933 total time=   0.5s
[CV 5/5; 1/10] START max_depth=1, n_estimators=300, reg_lambda=0.5..............
[CV 5/5; 1/10] END max_depth=1, n_estimators=300, reg_lambda=0.5;, score=0.967 total time=   0.0s
[CV 1/5; 2/10] START max_depth=7, n_estimato

## 3. Bayesian optimization

- 참고 : https://wooono.tistory.com/102

Bayesian Optimization 은 어느 입력값(x)를 받는 미지의 목적 함수 (f(x))를 판단하고 결정하여, 해당 함숫값 (f(x))을 최대로 만드는 최적해를 찾는 것을 목적으로 합니다.

즉, 목적 함수(탐색대상함수)와 하이퍼파라미터 쌍(pair)을 대상으로 Surrogate Model(대체 모델) 을 만들고,
순차적으로 하이퍼 파라미터를 업데이트해 가면서 평가를 통해 최적의 하이퍼파라미터 조합을 탐색합니다.
이 때의 목적 함수를 black-box function 이라고 합니다.
Bayesian Optimization 에는 두 가지 필수 요소가 존재합니다.

먼저 Surrogate Model 은, 현재까지 조사된 입력값-함숫결과값 점들 $(x_1, f(x_1)),...,(x_t, f(x_t))$ 을 바탕으로, 미지의 목적 함수의 형태에 대한 확률적인 추정을 수행하는 모델을 지칭합니다. 그리고 Acquisition Function 은, 목적 함수에 대한 현재까지의 확률적 추정 결과를 바탕으로, ‘최적 입력값을 찾는 데 있어 가장 유용할 만한’ 다음 입력값 후보를 추천해 주는 함수를 지칭합니다.

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb33tsP%2FbtraMpvxJG0%2FSn7uQK7k910IQ7cP3ZM9vk%2Fimg.png'>

- 대략적인 수행 과정

    - 위의 파란색 선은, 우리가 찾으려고 하는 목적함수 f(x) 를 나타내고,
    - 검정색 점선은, 지금까지 관측한 데이터를 바탕으로 우리가 예측한 estimated function 을 의미합니다.
    - 검정색 점선 주변에 있는 파란 영역은, 목적함수 f(x) 가 존재할만한 confidence bound(function의 variance) 를 의미합니다.
    - 밑에 있는 EI(x) 는, Acquisition function 을 의미하며, 다음 입력값 후보 추천 시 사용됩니다.
    - Acquisition function 값이 컸던 지점을 확인하고, 해당 지점의 hyperparameter 를 다음 입력 값으로 사용합니다.
    - hyperparamter 에 따라 estimated function 을 계속 update 하면, estimation function 과 목적 함수 f(x) 가 흡사해집니다.
    - 관측한 지점 중 best point 을 argmax f(x) 로 선택합니다.
- 자세한 수행 과정

    - 입력값, 목적 함수 및 그 외 설정값들을 정의합니다.

        - 입력값 x : 여러가지 hyperparameter
        - 목적 함수 f(x) : 설정한 입력값을 적용해 학습한, 딥러닝 모델의 성능 결과 수치(e.g. 정확도)
        - 입력값 x 의 탐색 대상 구간 : (a,b)
        - 입력값-함숫결과값 점들의 갯수 : n
        - 조사할 입력값-함숫결과값 점들의 갯수 : N
    - 설정한 탐색 대상 구간 (a,b) 내에서 처음 n 개의 입력값들을 랜덤하게 샘플링하여 선택합니다.

    - 선택한 n 개의 입력값 x1, x2, ..., xn 을 각각 모델의 hyperparameter 로 설정하여 딥러닝 모델을 학습한 뒤, 학습이 완료된 모델의 성능 결과 수치를 계산합니다.

        - 이들을 각각 함숫결과값 f(x1), f(x2), ..., f(xn) 으로 간주합니다.
입력값-함숫결과값 점들의 모음 (x1, f(x1)), (x2, f(x2)), ..., (xn, f(xn)) 에 대하여 Surrogate Model 로 확률적 추정을 수행합니다.

    - 조사된 입력값-함숫결과값 점들이 총 N 개에 도달할 때까지, 아래의 과정을 반복적으로 수행합니다.

        - 기존 입력값-함숫결과값 점들의 모음 (x1, f(x1)),(x2, f(x2)), ..., (xt, f(xt)) 에 대한 Surrogate Model 의 확률적 추정 결과를 바탕으로, 입력값 구간 (a,b) 내에서의 EI 의 값을 계산하고, 그 값이 가장 큰 점을 다음 입력값 후보 x1 로 선정합니다.
        - 다음 입력값 후보 x1 를 hyperparameter 로 설정하여 딥러닝 모델을 학습한 뒤, 학습이 완료된 모델의 성능 결과 수치를 계산하고, 이를 f(x1) 값으로 간주합니다.
        - 새로운 점 (x2, f(x2)) 을 기존 입력값-함숫결과값 점들의 모음에 추가하고, 갱신된 점들의 모음에 대하여 Surrogate Model 로 확률적 추정을 다시 수행합니다.
        - 총 N 개의 입력값-함숫결과값 점들에 대하여 확률적으로 추정된 목적 함수 결과물을 바탕으로, 평균 함수 μ(x) 을 최대로 만드는 최적해를 최종 선택합니다. 추후 해당값을 hyperparameter 로 사용하여 딥러닝 모델을 학습하면, 일반화 성능이 극대화된 모델을 얻을 수 있습니다.



grid search와 random search는 f(x)값에 관계없이 하이퍼 파라미터를 집어넣은 후에 최종 정확도를 비교하지만   
bayesian optimization은 기존 입력값과 f(x)값을 바탕으로 하이퍼 파라미터 집합과 일반화 성능을 모델링한 surrogate model을 생성하고   
acquisition function이 최대가 되는 surrogate model을 계속 다시 모델링하면서 추정값이 실제 목적함수에 근사하게 되고 근사한 함수에서 최적값을 찾을 수 있다.

In [ ]:
def black_box_function(x, y):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    return -x ** 2 - (y - 1) ** 2 + 1

In [ ]:
!pip install bayesian-optimization

In [ ]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'x': (2, 4), 'y': (-3, 3)}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=1,
)

- n_iter: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.

- init_points: How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

In [ ]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

In [ ]:
print(optimizer.max)

In [ ]:
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

In [ ]:
optimizer.set_bounds(new_bounds={"x": (-2, 3)})

optimizer.maximize(
    init_points=0,
    n_iter=5,
)


- xgboost 하이퍼 파라미터 최적화

In [ ]:
#import packages
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris

#Loading iris dataset from sklearn
iris = load_iris()

#independent feautres
X = iris.data

# target features
y = iris.target

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, stratify=y)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb

# MAPE Metric
def mean_absolute_percentage_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

# 탐색 대상 함수 (XGBRegressor)
def XGB_cv(max_depth,learning_rate, n_estimators, gamma
            ,min_child_weight, subsample
            ,colsample_bytree, silent=True, nthread=-1):

    # 모델 정의
    model = xgb.XGBRegressor(max_depth=int(max_depth),
                            learning_rate=learning_rate,
                            n_estimators=int(n_estimators),
                            gamma=gamma,
                            min_child_weight=min_child_weight,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree, 
                            nthread=nthread
                            )
    # 모델 훈련
    model.fit(X_train, y_train)

    # 예측값 출력
    y_pred= model.predict(X_test)

    # 각종 metric 계산
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # 오차 최적화로 사용할 metric 반환
    return r2

In [ ]:
#  bayesian-optimization 라이브러리의 BayesianOptimization 클래스 import
from bayes_opt import BayesianOptimization
import numpy as np

# 실험해보고자하는 hyperparameter 집합
pbounds = {'max_depth': (3, 7),
            'learning_rate': (0.01, 0.2),
            'n_estimators': (5000, 10000),
            'gamma': (0, 100),
            'min_child_weight': (0, 3),
            'subsample': (0.5, 1),
            'colsample_bytree' :(0.2, 1)
            }

# Bayesian optimization 객체 생성
# f : 탐색 대상 함수, pbounds : hyperparameter 집합
# verbose = 2 항상 출력, verbose = 1 최댓값일 때 출력, verbose = 0 출력 안함
# random_state : Bayesian Optimization 상의 랜덤성이 존재하는 부분을 통제 
bo=BayesianOptimization(f=XGB_cv, pbounds=pbounds, verbose=2, random_state=1 )    

# 메소드를 이용해 최대화 과정 수행
# init_points :  초기 Random Search 갯수
# n_iter : 반복 횟수 (몇개의 입력값-함숫값 점들을 확인할지! 많을 수록 정확한 값을 얻을 수 있다.)
# acq : Acquisition Function들 중 Expected Improvement(EI) 를 사용
# xi : exploration 강도 (기본값은 0.0)
bo.maximize(init_points=2, n_iter=10, acq='ei', xi=0.01)

# ‘iter’는 반복 회차, ‘target’은 목적 함수의 값, 나머지는 입력값을 나타냅니다. 
# 현재 회차 이전까지 조사된 함숫값들과 비교하여, 현재 회차에 최댓값이 얻어진 경우, 
# bayesian-optimization 라이브러리는 이를 자동으로 다른 색 글자로 표시하는 것을 확인할 수 있습니다

# 찾은 파라미터 값 확인
print(bo.max)

# 4.Hyperopt

- HyperOpt는 자동화된 하이퍼파라미터 튜닝 프레임워크로서, fmin()이라는 함수 안에는 3가지의 파라미터가 있다:

    - Objective Function: 최소화할 손실 함수
    - Domain Space: 탐색 범위. 베이지안 최적화에서는 이 범위가 각  하이퍼파라미터에 대해 통계 분포를 만들어낸다.
    - Optimization Algorithm : 최적의 조합을 찾기 위한 알고리즘

참고 : https://velog.io/@emseoyk/%ED%95%98%EC%9D%B4%ED%8D%BC%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0-%ED%8A%9C%EB%8B%9D

In [ ]:
!pip install scikit-optimize

In [ ]:
#importing packages 

from hyperopt import hp,fmin, tpe, Trials

from hyperopt.pyll.base import scope

from functools import partial

from skopt import space

from skopt import gp_minimize

#defining a method that will perfrom a 5 split cross validation over

#dataset and and will produce the optimum value of the accuracy

def optimize(params, x,y):

    clf = XGBClassifier(**params)

    kf = model_selection.StratifiedKFold(n_splits=5)

    accuracies = []

    for idx in kf.split(X=x,y=y):

        train_idx,test_idx = idx[0],idx[1]

        xtrain = x[train_idx]

        ytrain = y[train_idx]

        xtest = x[test_idx]

        ytest = y[test_idx]

        clf.fit(xtrain,ytrain)

        preds =  clf.predict(xtest)

        fold_acc = metrics.accuracy_score(ytest,preds)

        accuracies.append(fold_acc)

    return -1.0 * np.mean(accuracies)

#defining a set of values as hp for hyperparameters

param_space = {

    "max_depth" : scope.int(hp.quniform("max_depth",3,20, 1)) ,

    "min_child_weight" : scope.int(hp.quniform("min_child_weight",1,8, 1)),

    "n_estimators": scope.int(hp.quniform("n_estimators",100,1500,1)),

    'learning_rate': hp.uniform("learning_rate",0.01,1),

    'reg_lambda': hp.uniform("reg_lambda",0.01,1),

    'gamma': hp.uniform("gamma",0.01,1),

    'subsample': hp.uniform("subsample",0.01,1)

    }

#defiing optimization_fuction as partial and calling optimize within it

optimization_fuction = partial(optimize,x = X, y = y) 

trials = Trials()

#Getting the optimum values for hyperparameters

result = fmin(

    fn = optimization_fuction,

    space = param_space,

    algo = tpe.suggest,

    max_evals = 15,

    trials = trials

)

#Printing the best hyperparemeter set

print(result)

# 5. Optuna ****************

- Optuna는 ML 알고리즘의 하이퍼파라미터 튜닝을 자동화해주는 오픈소스 툴입니다. 유사한 툴로 Hyperopt가 있지만 사용성과 문서화, 시각화 제공 여부 등에서 Optuna의 손을 들어주는 경우가 많음.

- 하이퍼파라미터 튜닝에 쓰고 있는 최신 Automl 기법입니다.
- 빠르게 튜닝이 가능하다는 장점이 있음.
- 하이퍼파라미터 튜닝 방식을 지정할수 있다. -> 직관적인 api인 튜닝된 lightgbm도 제공해줍니다.

- 다른 라이브러리들에 비해 직관적인 장점이 있어 코딩하기 용이합니다.


- 거의 모든 ML/DL 프레임워크에서 사용 가능한 넓은 범용성을 가지고 있다.
간단하고 빠르다.
- 최신 동향의 다양한 최적화 알고리즘을 갖추고 있다.
- 병렬 처리가 가능하다.
- 간단한 메소드로 시각화가 가능하다.

- Optuna를 이해하기 위해서는 다음의 용어에 익숙해져야 한다.

    - Study: 목적 함수에 기반한 최적화
    - Trial: 목적함수 시행
- 쉽게 말해 study는 최적화를 하는 과정이고, trial은 다양한 조합으로 목적함수를 시행하는 횟수를 뜻한다.
- Study의 목적은 여러 번의 trial을 거쳐 최적의 하이퍼파라미터 조합을 찾는 것이라고 할 수 있겠다.

In [5]:
!pip install optuna

     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/380.1 kB ? eta -:--:--
   --------------------------------------- 380.1/380.1 kB 11.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/233.4 kB ? eta -:--:--
   --------------------------------------- 233.4/233.4 kB 13.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------- ----------------------- 0.9/2.1 MB 27.4 MB/s eta 0:00:01
   ----------------------------------- ---- 1.8/2.1 MB 23.4 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 16.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/145.3 kB ? eta -:--:--
   ---------------------------------------- 145.3/145.3 kB 9.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.3 kB ? eta -:--:--
   ----------------------

DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [7]:
#importing all the required packages
import optuna
from sklearn.datasets import load_iris
import sklearn.linear_model
import sklearn.metrics
from sklearn.model_selection import train_test_split

#Loading iris dataset from sklearn
iris = load_iris()

#independent feautres
X = iris.data

# target features
y = iris.target

In [8]:
import optuna
print(optuna.__version__)

3.6.1


In [10]:
#importing packages
import optuna
from functools import partial

#defining a method that will perfrom a 5 split cross validation over
#dataset and and will produce the optimum value of the accuracy
def optimize(trial, x,y):
    #parameter set is declare within function
    #suggest_uniform : 범위 내의 균일 분포 값을 선택.
    #suggest_discrete_uniform : 이산 균등 분포를 값으로 선택.
    #suggest_loguniform : 범위 내의 로그 함수 선상의 값을 선택.
    reg_lambda = trial.suggest_uniform('reg_lambda',0.01,1)
    
    n_estimators = trial.suggest_int('n_estimators',100,1500)
    
    max_depth = trial.suggest_int('max_depth',3,15)
    
    max_features = trial.suggest_uniform('max_features',0.01,1)
    
    clf = XGBClassifier(
    
    n_estimators= n_estimators,

    reg_lambda=reg_lambda,

    max_depth=max_depth,

    max_features= max_features)

    kf = model_selection.StratifiedKFold(n_splits=5)

    accuracies = []

    for idx in kf.split(X=x,y=y):

        train_idx,test_idx = idx[0],idx[1]

        xtrain = x[train_idx]

        ytrain = y[train_idx]

        xtest = x[test_idx]

        ytest = y[test_idx]

        clf.fit(xtrain,ytrain)

        preds =  clf.predict(xtest)

        fold_acc = metrics.accuracy_score(ytest,preds)

        accuracies.append(fold_acc)

    return -1.0 * np.mean(accuracies)

#defiing optimization_fuction as partial and calling optimize within it

optimization_fuction = partial(optimize,x = X, y = y) 

study = optuna.create_study(direction='minimize')

#Printing the best hyperparemeter set

study.optimize(optimization_fuction, n_trials=15)

[I 2024-04-12 10:13:19,366] A new study created in memory with name: no-name-9047ebc1-effb-4d85-8ca4-25d4855e2d51
C:\Users\chldy\AppData\Local\Temp\ipykernel_14372\3806663716.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  reg_lambda = trial.suggest_uniform('reg_lambda',0.01,1)
C:\Users\chldy\AppData\Local\Temp\ipykernel_14372\3806663716.py:18: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  max_features = trial.suggest_uniform('max_features',0.01,1)
c:\Users\chldy\anaconda3\envs\torch\lib\site-packages\xgboost\core.py:160: UserWarning: [10:13:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: 

- mimimize로 찾은 이유 : https://stackoverflow.com/questions/72193393/find-the-value-of-variables-to-maximize-return-of-function-in-python

In [11]:
#importing packages
import optuna
from functools import partial

#defining a method that will perfrom a 5 split cross validation over
#dataset and and will produce the optimum value of the accuracy
def optimize(trial, x,y):
    #parameter set is declare within function
    #suggest_uniform : 범위 내의 균일 분포 값을 선택.
    #suggest_discrete_uniform : 이산 균등 분포를 값으로 선택.
    #suggest_loguniform : 범위 내의 로그 함수 선상의 값을 선택.
    reg_lambda = trial.suggest_uniform('reg_lambda',0.01,1)
    
    n_estimators = trial.suggest_int('n_estimators',100,1500)
    
    max_depth = trial.suggest_int('max_depth',3,15)
    
    max_features = trial.suggest_uniform('max_features',0.01,1)
    
    clf = XGBClassifier(
    
    n_estimators= n_estimators,

    reg_lambda=reg_lambda,

    max_depth=max_depth,

    max_features= max_features)

    kf = model_selection.StratifiedKFold(n_splits=5)

    accuracies = []

    for idx in kf.split(X=x,y=y):

        train_idx,test_idx = idx[0],idx[1]

        xtrain = x[train_idx]

        ytrain = y[train_idx]

        xtest = x[test_idx]

        ytest = y[test_idx]

        clf.fit(xtrain,ytrain)

        preds =  clf.predict(xtest)

        fold_acc = metrics.accuracy_score(ytest,preds)

        accuracies.append(fold_acc)

    return np.mean(accuracies)

#defiing optimization_fuction as partial and calling optimize within it

optimization_fuction = partial(optimize,x = X, y = y) 

study = optuna.create_study(direction='maximize')

#Printing the best hyperparemeter set

study.optimize(optimization_fuction, n_trials=15)

[I 2024-04-12 10:16:57,904] A new study created in memory with name: no-name-7a146972-e00f-46d4-888b-441db1548c63
C:\Users\chldy\AppData\Local\Temp\ipykernel_14372\1062776656.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  reg_lambda = trial.suggest_uniform('reg_lambda',0.01,1)
C:\Users\chldy\AppData\Local\Temp\ipykernel_14372\1062776656.py:18: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  max_features = trial.suggest_uniform('max_features',0.01,1)
c:\Users\chldy\anaconda3\envs\torch\lib\site-packages\xgboost\core.py:160: UserWarning: [10:16:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: 

## Create an objective function

In [13]:
## In optuna, A Trial represents a single call of the objective function
## Study shows an optimization session which contains a set of trials
## Study: optimization based on an objective function
## Trial: a single execution of the objective function

## In this demo, "alpha" is the hyperparameter which is need to be optimized
def objective(trial):
   
    # hyperparameter setting, trial.suggest_uniform will suggest uniform hyperparameter
    #alpha between the range of 0.0 to 2.0, lowest value of interval is closed and 
    #when low=high, it will return low value
    alpha = trial.suggest_uniform('alpha', 0.0, 2.0)
    
    # data loading and train-test split
    X, y = load_iris(return_X_y=True)
    X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0)
    
    # model training and evaluation
    model = sklearn.linear_model.Lasso(alpha=alpha)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    error = sklearn.metrics.mean_squared_error(y_val, y_pred)

    # output: evaluation score
    return error

## Create an study for that ML model and optimize it

In [14]:
import sklearn
# In Optuna, we use the study object to manage optimization.
# Method :func:`~optuna.create_study` returns a study object.
# A study object has useful properties for analyzing the optimization outcome.
study = optuna.create_study(direction='minimize') #Set minimize for minimization and maximize for maximization.
#To start the optimization, we create a study object and pass the objective function to method
study.optimize(objective, n_trials=50)

[I 2024-04-12 10:17:35,806] A new study created in memory with name: no-name-754e209f-c23a-4eab-a0a8-dcc2644716cd
C:\Users\chldy\AppData\Local\Temp\ipykernel_14372\2118815928.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 2.0)
[I 2024-04-12 10:17:35,827] Trial 0 finished with value: 0.5877416756176155 and parameters: {'alpha': 1.7732367839449403}. Best is trial 0 with value: 0.5877416756176155.
C:\Users\chldy\AppData\Local\Temp\ipykernel_14372\2118815928.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 2.0)
[I 2024-04-12 10:17:35,832] Trial 1 finished with value: 0.13019142061832117 and parameters: {'alpha': 

## Visualized the above hyperparameter optimization study

In [15]:
#importing all the plot functions
from optuna.visualization import plot_edf
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [16]:
# Visualize the optimization history. See :func:`~optuna.visualization.plot_optimization_history` for the details.
plot_optimization_history(study)

In [17]:
# Visualize high-dimensional parameter relationships. See :func:`~optuna.visualization.plot_parallel_coordinate` for the details.
plot_parallel_coordinate(study)

In [18]:
# Visualize individual hyperparameters as slice plot. See :func:`~optuna.visualization.plot_slice` for the details.
plot_slice(study)

In [19]:
# Visualize parameter importances. See :func:`~optuna.visualization.plot_param_importances` for the details.
#In this case, we have only one parameter.
plot_param_importances(study)

In [20]:
# Visualize empirical distribution function. See :func:`~optuna.visualization.plot_edf` for the details.
plot_edf(study)

In [21]:
import optuna
import sklearn
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

# 1. 최소화/최대화할 목적함수 정의
def objective(trial):
    iris = sklearn.datasets.load_iris()
    x, y = iris.data, iris.target

# 2. trial object로 하이퍼파라미터 값 추천
# 다양한 분류모델을 설정해서 비교할 수 있다.
    classifier_name = trial.suggest_categorical('classifier', ['SVC', 'RandomForest'])
    #분류 모델이 SVC일 때
    if classifier_name == 'SVC':
        svc_c = trial.suggest_loguniform('svc_c', 1e-10, 1e10)
        classifier_obj = sklearn.svm.SVC(C=svc_c, gamma='auto')
    
    #분류모델이 랜덤포레스트일 때
    else:
        rf_max_depth = int(trial.suggest_loguniform('rf_max_depth', 2, 32))
        classifier_obj = sklearn.ensemble.RandomForestClassifier(max_depth=rf_max_depth, n_estimators=10)
    
    accuracy = cross_val_score(classifier_obj, x, y, cv = 4).mean()
    return accuracy

# 3. study 오브젝트 생성하고 목적함수 최적화하는 단계
# 여기서는 목적함수를 정확도로 설정했기 때문에 최대화를 목표로 하고 있지만, 손실함수의 경우 direction='minimize'로 설정
study = optuna.create_study(direction='maximize')
# 반복 시행 횟수(trial)는 200번으로
study.optimize(objective, n_trials=200)

[I 2024-04-12 10:18:19,032] A new study created in memory with name: no-name-16f6c029-eef1-40f2-9538-bf10b921e5b7
C:\Users\chldy\AppData\Local\Temp\ipykernel_14372\3750847957.py:16: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

[I 2024-04-12 10:18:19,046] Trial 0 finished with value: 0.4822190611664296 and parameters: {'classifier': 'SVC', 'svc_c': 0.00013242665204694652}. Best is trial 0 with value: 0.4822190611664296.
C:\Users\chldy\AppData\Local\Temp\ipykernel_14372\3750847957.py:16: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

[I 2024-04-12 10:18:19,057] Trial 1 finished with value: 0.4822190611664296 and parameters: {'classifier': 'SVC', 'svc_c': 2.307676967750783

In [22]:
# 시행된 trial 중 최적의 하이퍼파라미터 반환하는 메소드
print(study.best_trial.params)

# 시행된 trial 중 가장 높은 값 반환하는 메소드
optuna_acc = study.best_trial.value
print(optuna_acc)

{'classifier': 'SVC', 'svc_c': 8.250691978620889}
0.9735064011379801


## Visualization

In [23]:
optuna.visualization.plot_param_importances(study)

optuna.visualization.plot_optimization_history(study)

# 6.Pycaret

- AutoML을 하게 해주는 파이썬 라이브러리
- scikit-learning 패키지를 기반으로 하고 있으며 Classification, Regression, Clustering, Anomaly Detection 등 다양한 모델을 지원함.

- 공식문서에 설명이 매우 잘 되어 있고, 몇 줄의 코드로 쉽게 구현이 가능하기 때문에 유용하게 사용할 수 있음.

- Pycaret을 활용하면 여러 모델의 성능 비교 뿐만 아니라 hyperparameter tunning,  여러 모델을 blending한 모델을 만들 수 있습니다.

In [ ]:
#Pycaret 설치
!pip install pycaret

In [ ]:
#모델 비교시 catboost가 없다면 다음과 같이 설치합니다.
!pip install pycaret[full]

In [ ]:
#Data load / pycaret에서 제공하는 'credit'데이터를 사용함.
from pycaret.datasets import get_data
dataset = get_data('credit')

In [ ]:
#train test 분리
train = dataset.sample(frac=0.95, random_state=786)
test = dataset.drop(train.index)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

- 데이터 설정.
    - Pycaret을 사용하기 전에 Pycaret에 맞게 데이터를 설정해줘야 함.
    - set_up() 함수를 사용하며, 기본적으로 data와 target을 입력해줍니다.
    - 입력 후 column에 대한 자료형이 출력되며 enter를 치면 data가 설정됩니다.

- set_up(): pycaret을 사용하기 위한 data setting

- session_id: random_state와 같은 개념으로 같은 결과가 나올 수 있게 seed를 고정합니다.
- data: train 데이터를 입력합니다.
- target = target 변수 이름을 입력합니다.

In [ ]:
from pycaret.classification import *

exp_clf = setup(data = train, target = 'default', session_id=123)

**- 모델 비교**


    - 여러 모델을 적합하여 성능을 비교하는 단계

In [ ]:
best_model = compare_models()

- compare_models(): 다양한 모델 적합 후 성능 비교
    - fold: cross_validation의 fold를 지정 (default = 10)
    - sort: 정렬기준 지표 설정
    - n_select: 상위 n개의 모델 결과만 출력

- 모델 적합
    - 하나의 모델의 적합 결과는 보는 방법.

In [ ]:
rf = create_model('rf')

- create_model(): 하나의 모델 적합

    - fold: cross_validation의 fold 지정 (default = 10)

**- Tunning**


    - tune_model() 함수를 사용해서 모델의 하이퍼파라미터 튜닝을 진행

In [ ]:
tuned_rf = tune_model(rf)

- tuned_rf를 호출하면 튜닝 결과를 확인할 수 있습니다.

- tune_model(model): 모델의 하이퍼파라미터 튜닝

- optimize: 평가 metric 지정

**Blending**

- blend_models() 함수를 사용하면 여러 모델들을 혼합하여 새로운 모델을 생성합니다.
모델을 하나씩 생성해서 blend해도 되고 compare_model을 사용하여 생성한 모델을 사용해서 blend할 수 있습니다.

In [ ]:
# 방법 1
dt = create_model('dt')
rf = create_model('rf')

blender_2 = blend_models(estimator_list = [dt, rf])

# 방법 2
best_model_5 = compare_models(n_select=5)
blender_5 = blend_models(best_model_5)

- blend_models(models): 여러 모델들을 혼합한 새로운 모델을 생성

**- 예측**

- finalize_model() 함수로 모델을 설정하면, cross_validation을 사용하여 적합한 모델을 전체 데이터로 마지막으로 학습을 합니다. 마지막 모델을 설정한 후에 predict_model()을 통해 예측을 합니다.

In [ ]:
final_model = finalize_model(blender_5)
prediction = predict_model(final_model, data = test)

- finalize_model(): 최종 모델로 설정 후 마지막 학습 진행
- predict_model(): 예측 결과를 'Label' 변수에 저장

In [ ]:
!pip install numba --upgrade

In [ ]:
from pycaret.classification import *
data = setup(diabetes, target = 'Class variable')

4) 모델 비교

In [ ]:
compare_models()

5) 모델 생성

In [ ]:
lr = create_model('lr')

6) 모델 튜닝

In [ ]:
tuned_lr = tune_model(lr)

7) Bagging

In [ ]:
bagged_lr = ensemble_model(lr, method = 'Bagging')

8) Stacking

In [ ]:
lda = create_model('lda')
rf = create_model('rf')
stacker = stack_models(estimator_list = [lda, rf], meta_model = lr)

9) Plot

In [ ]:
plot_model(lr)

10) 모델 저장

In [ ]:
save_model(lr, 'lr_saved')

11) Model load

In [ ]:
lr_saved = load_model('lr_saved')

### 7. auto-sklearn(https://automl.github.io/auto-sklearn/master/index.html)

- scikit-learning은 다양한 ML estimator를 제공하고 있음.
    - ML 모델을 만들기 위해 scikit-learn을 사용할 때 흔히 고민에 빠지는 포인트가 있습니다.
        - 어떤 estimator를 골라야 하나?>
        - estimator의 파라미터는 뭘로 세팅을 해야 하나? 값은 어떻게 튜닝을 할까?
        - 이런 결정은 다른 사람들은 어떻게 할까?
- 소개 글
    - auto-sklearn is an automated machine learning toolkit and a drop-in replacement for a scikit-learn estimator.

- auto-sklearn의 가장 큰 장점 중 하나는 바로 scikit-learn과의 api 유사성으로 인해 scikit-learn 사용자에게 친숙한 인터페이스를 제공합니다. 이를 통해 classifier, regressor를 구현할 때 별도의 튜닝 포인트 없이 편하게 사용할 수 있습니다.

- 작동원리

<img src='https://github.com/automl/auto-sklearn/raw/master/doc/images/askl_pipeline.png'>


- auto-sklearn은 AutoML을 CASH 문제를 푸는 것으로 다루고 있습니다.

- CASH는 Combined Algorithm Selection and Hyperparameter optimization 의 약자로 한글로 풀면 알고리즘 선택과 하이퍼파라미터 튜닝을 의미합니다.

- 위 그림에서 auto-sklearn에서 추가한 단계는 meta-learning 과 ensemble 입니다

    - 1) meta-learning 단계는 Bayesian optimizer를 warm start 해주는 역할을 합니다. 

    - 이 단계를 통해 Bayesian optimizer를 좀더 효율적으로 돌릴 수 있게 됩니다. 

    - 2) Bayesian optimizer를 통해 하이퍼파라미터 튜닝을 진행합니다 

    - 3) 지금까지 테스트한 모델과 파라미터 조합을 Ensemble하여 최적의 조합을 찾습니다.

In [ ]:
!sudo apt-get install build-essential swig
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install auto-sklearn

for _ in range(3):
    try:
        import autosklearn.classification
        break
    except:
        pass
else:
    raise ImportError("failed to import from autosklearn")

In [ ]:
import sklearn.datasets
import sklearn.model_selection

# From OpenML: https://www.openml.org/d/31
X, y = sklearn.datasets.fetch_openml('credit-g', as_frame=True, return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
   X, y, random_state=42
)

- auto-sklearn을 사용하기 전에 scikit-learn에 포함된 SVC를 이용하면 다음과 같이 분류할 수 있습니다.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
# Create the estimator using the default parameters from sklearn
estimator_svc = SVC(
    C=1.0, kernel='rbf', gamma='scale', shrinking=True, tol=1e-3,
    cache_size=200, verbose=False, max_iter=-1, random_state=42, probability=True
)

# build and fit the pipeline
encoder = ColumnTransformer(transformers = [
    ('cat', OneHotEncoder(handle_unknown='ignore'), X.dtypes == "category"),
    ('cont', StandardScaler(), X.dtypes != "category"),
])

pipeline_svc = Pipeline([
    ('encoder', encoder),
    ('svc', estimator_svc),
])
pipeline_svc.fit(X_train, y_train)

# Score the model
prediction = pipeline_svc.predict_proba(X_test)
performance_svc = roc_auc_score(y_test, prediction[:, 1])
print(f"SVC performance is {performance_svc}")

- SVC를 사용할 경우 하이퍼파라미터 튜닝을 통해 적절한 파라미터를 찾아가는 과정이 필요합니다.

그러면 auto-sklearn을 이용하면 어떨까요?

In [ ]:
import autosklearn.classification
import autosklearn.metrics

# Create and train an ensemble with AutoML 
estimator_askl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=300,
    seed=42, 
    resampling_strategy='cv',
    metric=autosklearn.metrics.roc_auc,
)
# Auto-sklearn ingests the pandas dataframe and detects column types
estimator_askl.fit(X_train, y_train, dataset_name='credit-g')

# Score the model
prediction = estimator_askl.predict_proba(X_test)

performance_askl = roc_auc_score(y_test, prediction[:, 1])
print(f"Auto-Sklearn Classifier performance is {performance_askl}")

- 별도의 튜닝없이도 괜찮은 성능의 모델을 만들어볼 수 있었습니다. 

- auto-sklearn을 돌려서 나온 결과를 좀더 자세히 살펴보겠습니다.

In [ ]:
print(estimator_askl.sprint_statistics())

In [ ]:
estimator_askl.leaderboard()

- auto-sklearn이 sklearn을 완벽히 대체할 silver bullet 이라 생각하지는 않습니다. 

- 하지만 auto-sklearn은 scikit-learn 사용자들이 적당한 성능을 빠른 시간에 얻기 위해 고려해볼만한 좋은 후보라고 생각합니다. 

###8. TPOT(https://epistasislab.github.io/tpot/)

- Python에서 AutoML을 수행하기위한 오픈 소스 라이브러리

- 데이터 변환 및 기계 학습 알고리즘에 인기있는 Scikit-Learn 기계 학습 라이브러리를 사용

- 유전 프로그래밍 확률 적 글로벌 검색 절차를 사용

- 주어진 데이터 세트에 대한 최고 성능의 모델 파이프 라인을 효율적으로 검색

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fn5XGo%2Fbtq3yKfzKjO%2FY7C6WkOIfdnYqTAv0ckgxk%2Fimg.png'>

- TPOT tutorial on the Titanic dataset
    - The Titanic machine learning competition on Kaggle is one of the most popular beginner's competitions on the platform. We will use that competition here to demonstrate the implementation of TPOT.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
telescope_data=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data',header=None)

In [ ]:
telescope_data.head()

In [ ]:
telescope_data.columns = ['fLength', 'fWidth','fSize','fConc','fConcl','fAsym','fM3Long','fM3Trans','fAlpha','fDist','class']

In [ ]:
telescope_data.head()

In [ ]:
telescope_data.info()

In [ ]:
telescope_data.describe()

In [ ]:
telescope_data['class'].value_counts()

In [ ]:
telescope_shuffle=telescope_data.iloc[np.random.permutation(len(telescope_data))]
tele=telescope_shuffle.reset_index(drop=True)
tele.head()

In [ ]:
tele['class']=tele['class'].map({'g':0,'h':1})

tele.head()

In [ ]:
tele_class = tele['class'].values

In [ ]:
pd.isnull(tele).any()

In [ ]:
tele = tele.fillna(-999)

In [ ]:
from sklearn.model_selection import train_test_split
training_indices, validation_indices = training_indices, testing_indices = train_test_split(tele.index,
                                                                                            stratify = tele_class,
                                                                                            train_size=0.75, test_size=0.25)

In [ ]:
from tpot import TPOTClassifier
from tpot import TPOTRegressor

tpot = TPOTClassifier(generations=5,verbosity=2) 

tpot.fit(tele.drop('class',axis=1).loc[training_indices].values,
         tele.loc[training_indices,'class'].values)

In [ ]:
tpot.score(tele.drop('class',axis=1).loc[validation_indices].values,
           tele.loc[validation_indices, 'class'].values)

In [ ]:
tpot.export('tpot_MAGIC_Gamma_Telescope_pipeline.py')

### 9. H2O

- h2o는 java 기반의 머신러닝/AI 플랫폼인데요.

- 드라이버가 필요없이 간단한 설정만으로 인공지능을 학습하고 예측할 수 있는 소프트웨어

- 인공지능에 대한 지식이 부족하여도 좋은 모델을 누구나 이용할 수 있는 소프트웨어가 개발되었음.


In [ ]:
!pip install h2o
import h2o

- 다음으로, H2O 클러스터를 시작합니다.

- 이제 타이타닉 데이터셋을 로드하고, H2O 데이터 프레임으로 변환합니다. 그리고 outcome variable을 지정합니다.

In [ ]:
h2o.init()

In [ ]:
titanic_df = h2o.import_file("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
titanic_df["Survived"] = titanic_df["Survived"].asfactor()

In [ ]:
train, test = titanic_df.split_frame(ratios=[0.8], seed=1)

- 이제 H2O AutoML을 실행하여 최상의 모델을 생성함.

In [ ]:
from h2o.automl import H2OAutoML

aml = H2OAutoML(max_runtime_secs=30, seed=1)
aml.train(y="Survived", training_frame=train)

- 모델링이 완료되면, AutoML 리더보드를 확인하여 모델의 성능을 비교할 수 있습니다.

In [ ]:
aml.leaderboard.head()

- 가장 성능이 좋은 모델을 선택하고 test 데이터셋에서 모델의 성능을 평가합니다.

In [ ]:
best_model = aml.leader
predictions = best_model.predict(test)

이 밖에 AutoML을 할 수 있는 라이브러리들은 다양하게 존재합니다:

- TPOT
- FLAML
- EvalML
- AutoKeras
- Auto-ViML
- AutoGluon
- MLBox

[Hyperparam Tuning & AutoML](https://www.kaggle.com/code/sudalairajkumar/hyperparam-tuning-automl)

참고) 

### Numba
 - Python 및 Numpy 코드의 하위 집합을 빠른 기계 코드로 변환하는 오픈소스 Jit 컴파일러.

 - Just In Time 컴파일러를 사용해 파이썬 코드 내에서 일반 코드 및 Numpy를 아주 빠른 속도로 처리 가능한 기능을 제공.

- 공식문서에 설명
    - Numba makes Python code fast.
    - Numba is an open source JIT compiler that translates a subset of Python and NumPy code into fast machine code.

- Numbda 작동원리
    - 데커레이팅된 함수에 대한 파이썬 bytecode를 읽고 이를 함수의 입력 인수 유형에 대한 정보와 결합한다,.
    - 코드를 분석하고 최적화한 후, LLVM compiler library를 사용하여 함수의 machine code 버전을 만들고, 이를 사용자의 CPU 능력에 맞춤.
    - 이 complied된 버전이 앞으로 그 함수를 호출할 때마다 사용된다.

- Numbda 모듈이 모든 파이썬 코드를 최적화 해주는 것은 아니다. 일부 파이썬 코드와 Numpy 대해서만 작동하며 다른 모듈을 이용한 코드를 최적화 시켜주지는 못한다. ex)Numbda는 Pandas를 이해하지 못함. 특정 목적에 따라 충분히 활용할 수 있는 가치가 있는 모듈.

In [ ]:
from time import perf_counter
from numba import jit

# 일반적인 loop
def pure_sum(n):
    result = 0
    for i in range(0, n+1):
        result += i
    return result

# Numba 모듈 사용
@jit(nopython=True, cache=True)
def numba_sum(n):
    result = 0
    for i in range(0, n+1):
        result += i
    return result

# 시간 재기: 일반
start = perf_counter()
pure_sum(100000000)
print(perf_counter() - start)

# 시간 재기에 앞서 먼저 Compile을 해준다.
numba_sum(1)

# 시간 재기: Numba
start = perf_counter()
numba_sum(100000000)
print(perf_counter() - start)

- @jit 데커레이터의 모드
    - @jit 데커레이터는 nopython과 object라는 2가지 compilation 모드로 작동한다.
    - 위 예제에서 nopython=True를 통해 Numba에게 nopython 모드로 작동하라고 지시한 셈인데, 이 모드는 decorate된 function을 근본적으로 compile하여 Python Interpreter의 개입 없이 전체가 작동하도록 한다.
    - 만약 nopython 모드가 잘 작동하지 않을 경우, Numba은 object 모드를 통해 compile 할 수 있다. @jit(nopython=True)가 아닌 @jit이라고만 데커레이팅하면 이 모드가 작동하게 된다.
    - 본 모드에서는 Numba은 loop를 식별하여 machine code에서 compile하며 나머지는 Intereter code에서 compile하게 된다. 더 나은 성능을 기대한다면 이 모드가 아닌 nopython 모드를 사용해야 한다. 
    